# Libraries

In [1]:
import random

import numpy as np

import torch
import torch.nn as nn
import torchtext
import torch.nn.functional as F
#from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
from torchtext.legacy.data import Field, BucketIterator, Iterator, TabularDataset # Needed for running this on my laptop
import torch.optim as optim

# Meta variables

In [2]:
#device = torch.device('cuda:0')
device = torch.device('cpu')

my_data_directory = "../../data/" # where to store files
my_models_directory = "../../models/"

mini_testing = True
my_train_file = "mini_train.csv" if mini_testing == True else "train.csv"
my_test_file  = "mini_test.csv" if mini_testing == True else "test.csv"

# Dataloader

In [3]:
batch_size = 3 

In [4]:
def dataloader(directory  = my_data_directory,
               train_file = my_train_file,
               test_file  = my_test_file,
               batch      = batch_size):
    
    num_whitespacer = lambda x: [int(e) for e in x.split(" ")]
    
    SENTENCE = Field(lower = True, 
                     batch_first = True, 
                     init_token = "<sos>", 
                     eos_token = "<eos>")
    
    PREDICATE = Field(tokenize = num_whitespacer, # Here might be some problems ...
                      batch_first = True, 
                      pad_token = 0,
                      use_vocab = False,
                      init_token = 0, 
                      eos_token = 0) 
    
    SRLABEL = Field(batch_first = True, 
                    init_token = "<sos>", 
                    eos_token = "<eos>")
    
    my_fields = [("sentence", SENTENCE),
                 ("predicate", PREDICATE),
                 ("srlabel", SRLABEL)]
    
    train, test = TabularDataset.splits(path   = directory,
                                        train  = train_file,
                                        test   = test_file,
                                        format = 'csv',
                                        fields = my_fields,
                                        csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'¤'}) # Seems not to be in data
    SENTENCE.build_vocab(train)
    SRLABEL.build_vocab(train)  

    train_iter, test_iter = BucketIterator.splits((train, test),
                                                  batch_size        = batch,
                                                  sort_within_batch = True,
                                                  sort_key          = lambda x: len(x.sentence),
                                                  shuffle           = True,
                                                  device            = device)

    return train_iter, test_iter, SENTENCE.vocab, SRLABEL.vocab

# Model Definitions

In [5]:
class SRL_Encoder(nn.Module):
    def __init__(self, voc_size, embedding_size, hidden_size, n_layers, p_dropout):  
        super(SRL_Encoder, self).__init__()
        
        self.embeddings = nn.Embedding(voc_size, embedding_size)
        self.sp_pair = embedding_size + 1 # emedded sentence + predicate vector
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.rnn = nn.LSTM(self.sp_pair, 
                           self.hidden_size, 
                           num_layers = self.n_layers,
                           dropout = p_dropout,
                           #bidirectional=True, # !
                           batch_first=True) # !
        self.dropout = nn.Dropout(p_dropout)
        
    def forward(self, sentences, pred_vec):
        
        embeddings = self.embeddings(sentences)
        pred_vec = pred_vec.unsqueeze(2)        
        sentence_pred_pair = torch.cat((embeddings, pred_vec), dim=2)
        contextualized_embedding, (hidden_final, cell_final) = self.rnn(sentence_pred_pair)
        
        #print("ENC, hidden:", hidden_final.shape)
        #print("ENC, cell:", cell_final.shape)
        
        return hidden_final, cell_final


In [6]:
class SRL_Decoder(nn.Module):
    def __init__(self, n_labels, embedding_size, hidden_size, n_layers, p_dropout):  
        super(SRL_Decoder, self).__init__()
 
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_labels = n_labels
        
        self.embeddings = nn.Embedding(n_labels, embedding_size) # ?
        self.rnn = nn.LSTM(embedding_size, 
                           self.hidden_size, 
                           num_layers = self.n_layers, 
                           batch_first=True,
                           #bidirectional=True,
                           dropout = p_dropout)
        self.classifier = nn.Linear(hidden_size, self.n_labels)
        
    def forward(self, previous, hidden, cell):
        
        #previous = previous.unsqueeze(1)
        
        embedded = self.embeddings(previous)
        #print("DEC, emb_previous:", embedded.shape)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.classifier(output)
        
        return prediction, hidden, cell


In [9]:
class SRL_Seq2SeqLabeler(nn.Module):
    def __init__(self, encoder, decoder):  
        super(SRL_Seq2SeqLabeler, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
        assert encoder.hidden_size == decoder.hidden_size, "hidden dimension of encoder must be equal to that of decoder"
        assert encoder.n_layers == decoder.n_layers, "n_layers of encoder must be equal to that of decoder"
        
    def forward(self, sentence, predicate, srl_labels, tfr = None): # teacher forcing ratio (tfr)

        batch_size = sentence.shape[0]
        seq_len = sentence.shape[1]
        n_labels = self.decoder.n_labels

        outputs = torch.zeros(batch_size, seq_len, n_labels).to(device) # for storage

        hidden, cell = self.encoder(sentence, predicate)

        seq_element = srl_labels[:, 0].unsqueeze(1) # start of sentence token; index of <sos>
        #print("S2S, Seq_elem, prior:", seq_element)

        for l in range(1, seq_len): #start from 1; first column of outputs will "remain" 0
            
            #print("S2S, Seq_elem:", seq_element.shape)

            output, hidden, cell = self.decoder(seq_element, hidden, cell)
            #print("S2S, Outputs:", outputs.shape)
            #print("S2S, Output:", output.shape)
            outputs[:, l, :] = output.squeeze()
            best_guess = output.argmax(2)
            #print("S2S, Best guess:", best_guess)

            if tfr != None:
                teacher_force = random.random() < tfr
#                 if teacher_force:
#                     print("TF")
#                 else:
#                     print("No TF")
                seq_element = srl_labels[:, l].unsqueeze(1) if teacher_force else best_guess
            else:
                seq_element = best_guess

        return outputs

In [ ]:
random.random()

# Training function

In [18]:
def trainer(model, # Must be an instance of a model!
            name_of_model,
            learning_rate,
            epochs,
            data,
            my_tfr = 0.5,
            val_data = None,
            save_model = False,
            directory = my_models_directory,
            my_loss_function = nn.CrossEntropyLoss,
            my_optimizer = optim.Adam
           ):
    """ Specifices a general training procedure for a model. 
        Note: trainer() requires an instantiated model as model argument. 
    """
    
    optimizer = my_optimizer(model.parameters(), lr=learning_rate)    
    
    model.to(device)
    model.train()
    
    pad_idx = labels.stoi["<pad>"]
    
    loss_function = my_loss_function(ignore_index=pad_idx)
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        for i, batch in enumerate(data):
            optimizer.zero_grad # reset gradients
            
            sentence = batch.sentence
            predicate = batch.predicate
            targets = batch.srlabel
            #print("TRAIN, Targets:", targets.shape)
            
            output = model(sentence, predicate, targets, tfr = my_tfr)
            #print("TRAIN, Output:", output.shape)
            
            ### STUFF TO BE DONE WITH OUTPUT AND TARGETS HERE ###
            
            bsz = output.shape[0]
            length = output.shape[1]
            output_dim = output.shape[2]
        
            output = output[:, 1:, :].reshape(bsz*(length - 1), output_dim) # first token ("column") being zeroes
            targets = targets[:, 1:].flatten() # first token being <sos>

            loss = loss_function(output, targets)
            
            epoch_loss += loss.item()
            loss.backward() # compute gradients
            optimizer.step() # update parameters
            #break
            
        print(f"Epoch: {epoch+1} (out of {epochs}); total loss: {epoch_loss}.")
            
        if val_data != None:
            model.eval()
            # Here we could do some evaluation of model progress, but I have ignored this for now. 
            model.train()
            
    if save_model == True:
        torch.save(model, directory+name_of_model+".pt")

# Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
# https://www.baeldung.com/cs/multi-class-f1-score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html


def metrics(prediction, truth, labels):
    """ Calculates accuracy and F1.  
    """
    
    score_keeping = {}
    
    accuracy = accuracy_score(truth, prediction)
    f1_lsted = f1_score(truth, prediction, labels = labels, average = None)
    f1_macro = f1_score(truth, prediction, labels = labels, average = "macro") # Calculate metrics for each label, and find their unweighted mean. Does not take label imbalance into account.
    f1_micro = f1_score(truth, prediction, labels = labels, average = "micro") # Calculate metrics globally by counting the total true positives, false negatives and false positives.
    
    score_keeping["accuracy"] = accuracy
    score_keeping["f1_macro"] = f1_macro
    score_keeping["f1_micro"] = f1_micro
    for label in labels:
        score_keeping[label] = f1_lsted[labels.index(label)] # overkill?
    
    return score_keeping

In [ ]:
def mean(array):
    """ Calculates the mean and standard deviation of an aray of numbers.
    """
#     print(array)
    mean = np.mean(array)
    std  = np.std(array)
    return mean, std

In [ ]:
class Evaluation:
    """ For storing and handling information from the evaluation of model(s).
    """
    
    def __init__(self, name):
        self.name = name
        self.pooled_acc      = "Not yet defined"
        self.pooled_f1_macro = "Not yet defined"
        self.pooled_f1_micro = "Not yet defined"        
        self.mean_acc        = ("Not yet defined", "Not yet defined")
        self.mean_f1_macro   = ("Not yet defined", "Not yet defined")
        self.mean_f1_micro   = ("Not yet defined", "Not yet defined")
        self.corr_l_acc      = "Not yet defined"
        self.corr_l_f1_macro = "Not yet defined"
        self.corr_l_f1_micro = "Not yet defined"
        
        self.metrics_dict = {"accuracy": ["Not yet defined", "Not yet defined"], 
                             "f1_macro": ["Not yet defined", "Not yet defined"], 
                             "f1_micro": ["Not yet defined", "Not yet defined"]}

    def best_case(self, metric):
        """ Returns the file which has the best performance score with respect 
            to a metric.
        """
        m_list = self.metrics_dict[metric]
        zic_zac = False if metric == "mse" else True
        m_list.sort(key=operator.itemgetter(1), reverse=zic_zac)
        return m_list[0][0]
    
    def best_cases(self, metric, n):
        """ Returns a list of the N files which has the best performance score 
            with respect to a metric.
        """
        m_list = self.metrics_dict[metric]
        zic_zac = False if metric == "mse" else True
        m_list.sort(key=operator.itemgetter(1), reverse=zic_zac)
        files, values = zip(*m_list)
        return list(files[:n])
    
    def worst_case(self, metric):
        """ Returns the file which has the best performance score with respect 
            to a metric.
        """
        m_list = self.metrics_dict[metric]
        zic_zac = True if metric == "mse" else False
        m_list.sort(key=operator.itemgetter(1), reverse=zic_zac)
        return m_list[0][0]

    def worst_cases(self, metric, n):
        """ Returns a list of the N files which has the best performance score 
            with respect to a metric.
        """
        m_list = self.metrics_dict[metric]
        zic_zac = True if metric == "mse" else False
        m_list.sort(key=operator.itemgetter(1), reverse=zic_zac)
        files, values = zip(*m_list)
        return list(files[:n])
 
    def summary(self):
        """ Summarises an evaluation. Returns string."""
        summary  = "\n".join([f"Model {self.name} performs as follows:", 
                      f"Pooled Accuracy: {self.pooled_acc}",
                      f"Pooled F1_macro: {self.pooled_f1_macro}",
                      f"Pooled F1_micro: {self.pooled_f1_micro}",
                              
                      f"Mean Accuracy: {self.mean_acc[0]} (std = {self.mean_acc[1]})",
                      f"Mean F1_macro: {self.mean_f1_macro[0]} (std = {self.mean_f1_macro[1]})",
                      f"Mean F1_micro: {self.mean_f1_micro[0]} (std = {self.mean_f1_micro[1]})",
                      
                      f"Correlation sentence length and accuracy: {self.corr_l_acc}",
                      f"Correlation sentence length and F1_macro: {self.corr_l_f1_macro}",
                      f"Correlation sentence length and F1_micro: {self.corr_l_f1_micro}"]) 
        return summary
    
    def save(self, directory=path_to_save_evaluations):
        """ Writes the summary of an evaluation to a text file (at some diectory)."""
        
        summary = self.summary()
        with open(directory+self.name+".txt", "w") as e:
            e.write(summary)
    
    def print_summary(self):
        """ Prints out the summary of an evaluation.
        """
        summary = self.summary()
        print(summary)

In [ ]:
# which sentence-predicate pairs are best, worst
# length? correlation of length and [F1, accuracy]
# confusion

def evaluator(model, name, test_data = test, labels = lst_labels, detach_me=False):
    """ 
    """
    t1 = time.perf_counter()
    
    model.eval()
    
    evaluation = Evaluation(name)
    
    prediction_pooled = []
    truth_pooled = []
    seq_lengths = []
    confusion = {label: {label: 0 for label in labels} for label in labels}
    metrics_calc = {"accuracy": [], "f1_macro": [], "f1_micro": []}
    
    #i=1 # in order to print out progress
    for batch in test_data:

        sentence = batch.sentence
        predicate = batch.predicate
            
        if detach_me == True:
            prediction = model(sentence, predicate, trf = None).detach().to("cpu") 
            truth = batch.srlabel.detach().to("cpu")
        
        else:
            prediction = model(sentence, predicate, trf = None) 
            truth = batch.srlabel
        
        batched_pred_labels = prediction[:, 1:, :].argmax(2) # ... should not be batch-wise, but sentence-wise
        batched_true_labels = truth[:, 1:]
        
        bsz = batched_pred_labels.shape[0]
        
        for b in range(bsz):
            str_sent    = " ".join([vocab.itos[token] for token in sentence[b]]) # to list?
            seq_len     = len([x for x in sentence[b] if vocab.itos[x] not in ["<pad>", "<sos>", "<eos>"]])
            pred_labels = batched_pred_labels[b]
            true_labels = batched_true_labels[b]
            
            accuracy, f1_macro, f1_micro, X = metrics(true_labels, pred_labels)
            
            prediction_pooled.append(pred_labels)
            truth_pooled.append(true_labels)
            seq_lengths.append(seq_len)
            
            for p, t in zip(pred_labels, true_labels):
                confusion[p][t] += 1
                
            for m, v in zip(["accuracy", "f1_macro", "f1_micro"], [accuracy, f1_macro, f1_micro]):
                metrics_calc[m].append( (str_sent, v) )
                
#################################                
            
        pooled_accuracy, pooled_f1_macro, pooled_f1_micro, X = metrics(truth_pooled, prediction_pooled)
        
        mean_accuracy = mean(list(zip(*metrics_calc["accuracy"]))[1])
        mean_f1_macro = mean(list(zip(*metrics_calc["f1_macro"]))[1])
        mean_f1_micro = mean(list(zip(*metrics_calc["f1_micro"]))[1])
        
        
        
        
        mean(list(zip(*metrics_calc["accuracy"]))[1])
        
        
        
        
        
        
        
        
        bsz = prediction.shape[0]
        length = prediction.shape[1]
        output_dim = prediction.shape[2]
        
        

        prediction = prediction[:, 1:, :].reshape(bsz*(length - 1), output_dim) # first token ("column") being zeroes
        turth = targets[:, 1:].flatten() # first token being <sos>        
    
    
    
    
#     thld_frequencies_pooled = {"tp": 0, "fp": 0, "tn": 0, "fn": 0}
#     thld_metrics_calc = {"mse": [], "accuracy": [], "recall": [], "precision": [], "f1": []}

    i=1 # in order to print out progress
    for instance in test_data:
        
        if detach_me == True:
            prediction = torch.flatten(model(instance["img_vector"])).detach().to("cpu") # due to design of model
            truth = torch.flatten(instance["label"]).int().detach().to("cpu")
            file = instance["file"]
        
        else:
            prediction = torch.flatten(model(instance["img_vector"])) # due to design of model
            truth = torch.flatten(instance["label"]).int()
            file = instance["file"]
            
        prediction_pooled.append( (file, prediction) )
        truth_pooled.append(truth)        
        
        mse = F.mse_loss(prediction, truth)
        thld_metrics_calc["mse"].append( (file, mse.item()) )
        
        #######
        #roundof = (prediction >= 0.1).int()
        #print("sum of roundof, file:", file, torch.sum(roundof))
        #break
        #######
        
        # IF-block for calculating accuracy, recall, precision, f1, which is very
        # time consuming, due to identifications of TPs, FPs, TNs, and FNs for 
        # large matrices.
        if threshold != None:
            roundof = (prediction >= threshold).int()
            #print("sum of roundof", torch.sum(roundof))
            tp = sum(roundof * truth)
            fp = sum(roundof * (~truth.bool()))
            tn = sum((~roundof.bool()) * (~truth.bool()))
            fn = sum((~roundof.bool()) * truth)

            accuracy, recall, precision, f1 = thld_metrics(tp, fp, tn, fn)

            for key, value in zip(["tp", "fp", "tn", "fn"], [tp, fp, tn, fn]):
                thld_frequencies_pooled[key]+=value

            for key, value in zip(["accuracy", "recall", "precision", "f1"], 
                                  [accuracy.item(), recall.item(), precision.item(), f1.item()]):
                thld_metrics_calc[key].append( (file, value) )
        
        print("({}%)".format(round((i/len(test_data)*100), 1)), end="\r")
        i+=1
    
    file, predictions = zip(*prediction_pooled)
    evaluation.pooled_mse = F.mse_loss(torch.flatten(torch.stack(list(predictions))), 
                                       torch.flatten(torch.stack(truth_pooled))).item()
    
    if threshold != None:
        pooled_accuracy, pooled_recall, pooled_precision, pooled_f1 = thld_metrics(
            thld_frequencies_pooled["tp"], 
            thld_frequencies_pooled["fp"], 
            thld_frequencies_pooled["tn"], 
            thld_frequencies_pooled["fn"])
        
        evaluation.pooled_acc = pooled_accuracy
        evaluation.pooled_rec = pooled_recall
        evaluation.pooled_prc = pooled_precision
        evaluation.pooled_f1  = pooled_f1
    
    # The code below is a bit nested. What it does in plain English is:
    # go to the dictionary where we keep all the performance scores with respect to 
    # each file. Every key (i.e. metric) of that dict maps to a list of tupples of 
    # file and value of the metric. Here, we "unzip" that list of tupples and calculate the 
    # mean (and standard deviation) for the values and use that mean (and std) to define 
    # the respective attributes of the Evaluation class instance.
    
    evaluation.mean_mse = mean(list(zip(*thld_metrics_calc["mse"]))[1])
    
    if threshold != None:
        evaluation.mean_acc = mean(list(zip(*thld_metrics_calc["accuracy"]))[1]) 
        evaluation.mean_rec = mean(list(zip(*thld_metrics_calc["recall"]))[1]) 
        evaluation.mean_prc = mean(list(zip(*thld_metrics_calc["precision"]))[1]) 
        evaluation.mean_f1  = mean(list(zip(*thld_metrics_calc["f1"]))[1]) 
    
    evaluation.metrics_dict = thld_metrics_calc
    
    t2 = time.perf_counter()
    passed_time = t2 - t1
    print("Done! ({} m., {} s.)".format(int(passed_time/60), int(passed_time%60)))
    
    return evaluation, dict(prediction_pooled)

# Runnig it

## Data

In [11]:
train, test, vocab, labels = dataloader()

In [48]:
lst_labels = [labels.itos[x] for x in range(len(labels))]

In [49]:
lst_labels #remove stuff?

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '0',
 'I-ARG1',
 'I-ARG2',
 'B-PRD',
 'B-ARG1',
 'I-ARGM-MNR',
 'I-ARG0',
 'B-ARG0',
 'I-ARGM-ADV',
 'B-ARG2',
 'I-ARGM-PRP',
 'I-ARGM-TMP',
 'B-ARGM-MNR',
 'B-ARGM-TMP',
 'I-ARGM-LOC',
 'B-ARGM-MOD',
 'I-ARGM-DIR',
 'B-ARGM-ADV',
 'B-ARGM-LOC',
 'B-ARGM-NEG',
 'B-ARGM-PRP',
 'B-R-ARG1',
 'I-ARGM-PNC',
 'B-ARG4',
 'B-ARGM-CAU',
 'B-ARGM-DIR',
 'B-ARGM-DIS',
 'B-ARGM-PNC',
 'B-R-ARG0',
 'I-ARG4']

In [ ]:
vocab.stoi["<pad>"]

In [ ]:
labels.stoi["<pad>"]

## Training models

In [19]:
my_epochs = 3

vocab_size = len(vocab)
num_labels = len(labels)
emb_sizeE = 50
emb_sizeD = int(num_labels/2) # embeddings for labels
hid_size_encoder = 100
hid_size_decoder = hid_size_encoder # ?

num_layers = 2

my_encoder = SRL_Encoder(vocab_size, emb_sizeE, hid_size_encoder, num_layers, p_dropout=0.2)
my_decoder = SRL_Decoder(num_labels, emb_sizeD, hid_size_decoder, num_layers, p_dropout=0.2)

In [ ]:
my_encoder.parameters

In [ ]:
my_decoder.parameters

In [20]:
my_SRLLabeler = SRL_Seq2SeqLabeler(my_encoder, my_decoder)

In [22]:
model_name = f"srl_b{batch_size}_e{my_epochs}"
trainer(model = my_SRLLabeler, 
        name_of_model = model_name, 
        learning_rate = 0.01, 
        epochs = my_epochs, 
        data = train)

Epoch: 1 (out of 3); total loss: 65.64285123348236.
Epoch: 2 (out of 3); total loss: 96.61577893793583.
Epoch: 3 (out of 3); total loss: 79.67333137989044.


# Model 1

In [ ]:
class SRLabeler1(nn.Module):
    def __init__(self, voc_size, embedding_size, n_labels):  
        super(SRLabeler1, self).__init__()
        
        self.embeddings = nn.Embedding(voc_size, embedding_size)
        self.sp_pair = embedding_size + 1 # emedded sentence + predicate vector
        self.rnn = nn.LSTM(self.sp_pair, n_labels, bidirectional=True, batch_first=True)
        
    def forward(self, sentences, pred_vec, softmax=False):
        
        embeddings = self.embeddings(sentences)
        pred_vec = pred_vec.unsqueeze(2)        
        sentence_pred_pair = torch.cat((embeddings, pred_vec), dim=2)
        contextualized_embedding, *_ = self.rnn(sentence_pred_pair)
        
        if softmax == True:
            return F.softmax(contextualized_embedding, dim=2)
        else:
            return contextualized_embedding

Training

In [ ]:
def trainer1(model, # Must be an instance of a model!
            name_of_model,
            learning_rate,
            epochs,
            data,
            val_data = None,
            save_model = False,
            directory = my_models_directory,
            my_loss_function = nn.CrossEntropyLoss,
            my_optimizer = optim.Adam
           ):
    """ Specifices a general training procedure for a model. 
        Note: trainer() requires an instantiated model as model argument. 
    """
    
    optimizer = my_optimizer(model.parameters(), lr=learning_rate)    
    
    model.to(device)
    model.train()
    
    loss_function = my_loss_function()
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        for i, batch in enumerate(data):
            optimizer.zero_grad # reset gradients
            
            sentence = batch.sentence
            predicate = batch.predicate
            targets = batch.srlabel
            
            b=sentence.shape[0] # !
            sequence_length = sentence.shape[1] # !
            l = targets.shape[1] # !
                        
            output = model(sentence, predicate)
            d = output.shape[2] # !
            
            #print("Output:", output.shape)
            #print("Target:", targets.shape)
            
            loss = loss_function(output.reshape(b*sequence_length, d), # !
                                 targets.reshape(b*sequence_length))
            
            epoch_loss += loss.item()
            loss.backward() # compute gradients
            optimizer.step() # update parameters
            #break
            
        print(f"Epoch: {epoch+1} (out of {epochs}); total loss: {epoch_loss}.")
            
        if val_data != None:
            model.eval()
            # Here we could do some evaluation of model progress, but I have ignored this for now. 
            model.train()
            
    if save_model == True:
        torch.save(model, directory+name_of_model+".pt")

In [ ]:
vocab_size = len(vocab)
number_of_labels = len(labels)
print("Size of vocabulary:", vocab_size)
print("Number of labels:", number_of_labels)
epochs = 100
my_learning_rate = 0.01
my_emedding_size = 50
simple_srl_model = SRLabeler1(vocab_size, my_emedding_size, number_of_labels)
trainer(simple_srl_model, "simple_srl", my_learning_rate, epochs, train)

# Know your enemies; keep until ...

In [ ]:

for x in train:
    output = my_model(x.sentence, x.predicate)
    print("op", output.shape)
    soft = F.softmax(output, dim=2)
    print(torch.argmax(soft, dim=2))
    #print("sm", soft.shape)
    #print(torch.sum(soft, dim=2).shape)
